In [1]:
import numpy as np
import pandas as pd

In [2]:
from collections import Counter

acc_wins = Counter()
acc_fails = Counter()

In [3]:
with open('/Users/jilljenn/code/TF-recomm/data/0910_c_train.csv') as f:
    train_lines = f.read().splitlines()
with open('/Users/jilljenn/code/TF-recomm/data/0910_c_test.csv') as f:
    test_lines = f.read().splitlines()

In [4]:
def lines_to_df(lines, start_id=0):
    nb_users = len(lines) // 3
    user_ids = []
    item_ids = []
    outcomes = []
    nb_wins = []
    nb_fails = []
    for user_id in range(nb_users):
        acc_wins.clear()
        acc_fails.clear()
        nb = int(lines[3 * user_id])
        user_item_ids = list(map(int, lines[3 * user_id + 1].split(',')))
        user_outcomes = list(map(int, lines[3 * user_id + 2].split(',')))
        user_ids.extend([start_id + user_id] * nb)
        item_ids.extend(user_item_ids)
        outcomes.extend(user_outcomes)
        for item_id, outcome in zip(user_item_ids, user_outcomes):
            nb_wins.append(acc_wins[item_id])
            nb_fails.append(acc_fails[item_id])
            if outcome == 1:
                acc_wins[item_id] += 1
            else:
                acc_fails[item_id] += 1
    return nb_users, pd.DataFrame(np.column_stack((user_ids, item_ids, outcomes, nb_wins, nb_fails)), columns=('user_id', 'item_id', 'outcome', 'wins', 'fails'))

In [5]:
nb_users, train = lines_to_df(train_lines)

In [6]:
nb_users
#test = lines_to_df(test_lines)

3372

In [7]:
nb_test_users, test = lines_to_df(test_lines, start_id=nb_users)

In [8]:
test.tail()

,user_id,item_id,outcome,wins,fails
52418,4215,26,1,3,5
52419,4215,27,0,0,5
52420,4215,93,0,0,0
52421,4215,93,0,0,1
52422,4215,27,1,0,6


In [9]:
from sklearn.model_selection import train_test_split

subtrain, val = train_test_split(train, test_size=0.1)

In [10]:
subtrain.to_csv('/tmp/train.csv', header=False, index=False)
val.to_csv('/tmp/val.csv', header=False, index=False)

In [11]:
test.to_csv('/tmp/test.csv', header=False, index=False)

In [12]:
import yaml

with open('/tmp/config.yml', 'w') as f:
    config = {
        'USER_NUM': 1 + test['user_id'].max(),
        'ITEM_NUM': 1 + test['item_id'].max(),
        'NB_CLASSES': 2,
        'BATCH_SIZE': len(subtrain)
    }
    f.write(yaml.dump(config, default_flow_style=False))

In [13]:
test['item_id'].min(), test['item_id'].max(), train['item_id'].min(), train['item_id'].max()

(0, 145, 0, 145)

In [15]:
len(subtrain), len(val), len(test)

(200477, 22276, 52423)